In [1]:
cd /home/drew/NBOH/di-S-bond_ECL3/6WGTTmplt4EXL3/Desmond/Hirshfeld/retry/blockMembraneBackdoor/retry/ATB

/scr/drew/Serotonin/6WHA/equilibration/tryIonicCapsInSolvent/di-S-bond_ECL3/6WGTTmplt4EXL3/Desmond/Hirshfeld/retry/blockMembraneBackdoor/retry/ATB


In [11]:
import bioinf as bi
import numpy as np
import sys

def is_in_box(r, rmin, rmax):
    return np.all(np.greater(r,rmin) * np.less(r,rmax))

with open('every2nsFullSystem.pdb') as f, open('every2nsTrimmed.pdb', 'w') as o:
    model = 0
    min_ray = np.empty(3)
    max_ray = np.empty(3)
    atoms = np.array([])
    old_resSeq = 0
    for line in f:
        cleanline = line.strip()
        if cleanline.startswith('MODEL'):
            _,model_str = cleanline.split()
            model = int(model_str)
            min_ray = np.full(3, sys.float_info.max)
            max_ray = np.full(3, sys.float_info.min)
            
        elif cleanline.startswith('ATOM'):
            atom_line = bi.PDBAtomLine.parse_string(cleanline)
            atom = bi.PDBAtom(atom_line)
            if atom_line.resName == 'ASPH':
                atom_line = atom_line.copy_with(resName='ASP')
            elif atom_line.resName.startswith('HIS'):
                atom_line = atom_line.copy_with(resName='HIS')
            if atom_line.chainID == "A":
                min_ray = np.minimum(min_ray, atom.r)
                max_ray = np.maximum(max_ray, atom.r)
                o.write(f'{str(atom_line)}\n')
            else:
                new_resName = atom_line.resName
                if len(atom_line.resName) > 3:
                    new_resName = 'U0G' if atom_line.resName == 'CMYH' else "DPC"
                atom_line = atom_line.copy_with(kind='HETATM', chainID='', resName=new_resName)
                if atom_line.resSeq == old_resSeq:
                    atoms = np.append(atoms, bi.PDBAtom(atom_line))
                else:
                    rs = np.array([is_in_box(atom.r, min_ray, max_ray) for atom in atoms])
                    if np.any(rs):
                        for atom in atoms:
                            o.write(f'{str(atom)}\n')
                    atoms = np.array([bi.PDBAtom(atom_line)])
            old_resSeq = atom_line.resSeq
            continue
        o.write(f'{cleanline}\n')
                    
                    
            

In [16]:
atom.r

array(['46.210', '51.970', '68.970'], dtype='<U6')

In [8]:
atoms

array([], dtype=float64)